In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os

In [3]:
def get_csv_data(path_logs, path_csv):
    data = pd.read_csv(path_csv, sep='\t')
    dfs = []
    for dirname, _, filenames in os.walk(path_logs):
        for filename in filenames:
            if filename.endswith('.csv'):
                file_path = os.path.join(dirname, filename).replace('\\', '/')
                df = pd.read_csv(file_path,sep=' ')
                
                row = data.loc[data['log_id'] == int(filename[:-4])] # https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
                # print(row)
                arr = row.values.tolist()
                arr.append(df)
                dfs.append(arr)
    return dfs

In [4]:
dfs = get_csv_data('../the-attentive-cursor-dataset/logs', '../the-attentive-cursor-dataset/groundtruth.tsv')

In [5]:
cleandf = []
for row in range(len(dfs)):
    cleandf.append([dfs[row][0][2],dfs[row][1]['event'].value_counts()])

print(cleandf)

[[5, scroll          61
mousemove       22
mouseover        9
mousedown        6
mouseup          5
blur             3
focus            3
click            2
load             1
contextmenu      1
beforeunload     1
unload           1
Name: event, dtype: int64], [2, mousemove       9
scroll          7
mouseover       3
load            1
mousedown       1
mouseup         1
click           1
beforeunload    1
Name: event, dtype: int64], [4, mousemove       39
scroll          11
mouseover        3
blur             2
load             1
focus            1
beforeunload     1
unload           1
Name: event, dtype: int64], [2, mousemove       33
scroll          17
mouseover       13
blur             4
focus            4
load             1
mousedown        1
mouseup          1
click            1
beforeunload     1
unload           1
Name: event, dtype: int64], [4, mousemove       9
mouseover       3
blur            2
focus           2
load            1
beforeunload    1
unload          1
Name: ev

In [6]:
tab = np.array(cleandf, dtype='object')
for row in tab :
    for input in ['mouseup','focus','load','beforeunload','unload','touchend'] :
        if input in row[1].keys() :
            row[1].pop(input)

all_inputs = []
for row in tab :
    for input in row[1].keys() :
        if input not in all_inputs :
            all_inputs.append(input)

In [18]:
tab
train = tab[0 : int((len(tab)/100)*75)]
test = tab[int(len(tab)/100*75) : len(tab)-1]
tmp_x_train = []
tmp_x_test = []
tmp_y_train = []
tmp_y_test = []

for row in train :
    tmp_x_train.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_x_train[len(tmp_x_train)-1].append(row[1].get(input))
        else :
            tmp_x_train[len(tmp_x_train)-1].append(0)
    tmp_y_train.append(row[0])

for row in test :
    tmp_x_test.append([])
    for input in all_inputs :
        if input in row[1].keys() :
            tmp_x_test[len(tmp_x_test)-1].append(row[1].get(input))
        else :
            tmp_x_test[len(tmp_x_test)-1].append(0)
    tmp_y_test.append(row[0])

x_train = np.array(tmp_x_train)
x_test = np.array(tmp_x_test)
y_train = np.array(tmp_y_train)
y_test = np.array(tmp_y_test)
tmp_all_x = tmp_x_train
tmp_all_x.extend(tmp_x_test)
tmp_all_y = tmp_y_train
tmp_all_y.extend(tmp_y_test)
all_x = np.array(tmp_all_x)
all_y = np.array(tmp_all_y)
print(all_y)

[5 2 4 ... 4 3 4]


In [29]:
inputs = keras.Input(shape=(len(all_inputs),))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(6, activation=tf.nn.softmax)(x)

model = keras.Model(inputs=inputs, outputs = outputs)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(all_x, all_y, batch_size=len(x_train), epochs=100, validation_split=0.1)

Epoch 1/100
2/2 [==============================] - 4s 962ms/step - loss: 1.7510 - accuracy: 0.2859 - val_loss: 1.7556 - val_accuracy: 0.3548
Epoch 2/100
2/2 [==============================] - 0s 64ms/step - loss: 1.7476 - accuracy: 0.3362 - val_loss: 1.7548 - val_accuracy: 0.2774
Epoch 3/100
2/2 [==============================] - 0s 100ms/step - loss: 1.7456 - accuracy: 0.3211 - val_loss: 1.7540 - val_accuracy: 0.2774
Epoch 4/100
2/2 [==============================] - 0s 64ms/step - loss: 1.7441 - accuracy: 0.3218 - val_loss: 1.7533 - val_accuracy: 0.2839
Epoch 5/100
2/2 [==============================] - 0s 64ms/step - loss: 1.7427 - accuracy: 0.3240 - val_loss: 1.7526 - val_accuracy: 0.3613
Epoch 6/100
2/2 [==============================] - 0s 72ms/step - loss: 1.7414 - accuracy: 0.3427 - val_loss: 1.7519 - val_accuracy: 0.3613
Epoch 7/100
2/2 [==============================] - 0s 72ms/step - loss: 1.7403 - accuracy: 0.3441 - val_loss: 1.7512 - val_accuracy: 0.3613
Epoch 8/100
2/2 [=